## Day 4: Giant Squid

In [1]:
from typing import List
from pathlib import Path
import re


In [3]:
with Path('./4-test-input.txt') as file:
    bingo_text = file.read_text()
    for line in bingo_text.splitlines():
        print(line)
# bingo_text.splitlines()

7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7


In [76]:
class Board:
    def __init__(self, lines: List[str]) -> None:
        self.board: List[List[int]] = [
            [(int(n), False) for n in re.findall('\d+', line)] for line in lines]

    def __repr__(self) -> str:
        return f"Board: {self.board}"

    @property
    def winner(self):
        for row in self.board:
            if sum(n[1] for n in row) >= 5:
                return True
        for col in tuple(zip(*self.board)):
            if sum(n[1] for n in col) >= 5:
                return True
        return False

    @property
    def sum_unmarked(self):
        return sum([value[0] for row in self.board for value in row if not value[1]])

    def mark(self, number):
        for y, row in enumerate(self.board):
            for x, value in enumerate(row):
                if value[0] == number:
                    self.board[y][x] = (number, True)
                    return self.winner
        return self.winner


b = Board("""14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7""".splitlines())

In [78]:
b.sum_unmarked

325

In [79]:
b.mark(2)
b.mark(0)
b.mark(12)
b.mark(3)
# b.mark(7)
print(b.winner)
print(b.sum_unmarked)
b

False
308


Board: [[(14, False), (21, False), (17, False), (24, False), (4, False)], [(10, False), (16, False), (15, False), (9, False), (19, False)], [(18, False), (8, False), (23, False), (26, False), (20, False)], [(22, False), (11, False), (13, False), (6, False), (5, False)], [(2, True), (0, True), (12, True), (3, True), (7, False)]]

In [98]:
class Bingo:
    def __init__(self, input: str) -> None:
        self.lines = input.splitlines()
        self.numbers: List[int] = [int(i) for i in self.lines[0].split(',')]
        self.boards = [Board(self.lines[n*6+2:n*6+7])
                       for n in range(int(len(self.lines)/6))]
        self.winners = []

    @classmethod
    def from_file(cls, filename: str):
        with Path(filename) as file:
            bingo_text = file.read_text()
        return cls(bingo_text)
        
    def draw(self):
        for n in self.numbers:
            winner_index = [i for i, board in enumerate(self.boards) if board.mark(n)]
            for i in sorted(winner_index, reverse=True):
                self.winners.append((self.boards.pop(i), n))
            yield self.winners

    def draw_until_win(self):
        for winners in self.draw():
            if winners:
                return winners[0]
        return []

    def draw_until_last(self):
        for winners in self.draw():
            pass
        return winners[-1]


bingo_test = Bingo(bingo_text)
print(bingo_test.numbers)
winner, number = bingo_test.draw_until_win()
print(bingo_test.boards)

print("Winner: ", winner)
print(winner.sum_unmarked, number, number*winner.sum_unmarked)


[7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, 13, 6, 15, 25, 12, 22, 18, 20, 8, 19, 3, 26, 1]
[Board: [[(22, False), (13, False), (17, True), (11, True), (0, True)], [(8, False), (2, True), (23, True), (4, True), (24, True)], [(21, True), (9, True), (14, True), (16, False), (7, True)], [(6, False), (10, False), (3, False), (18, False), (5, True)], [(1, False), (12, False), (20, False), (15, False), (19, False)]], Board: [[(3, False), (15, False), (0, True), (2, True), (22, False)], [(9, True), (18, False), (13, False), (17, True), (5, True)], [(19, False), (8, False), (7, True), (25, False), (23, True)], [(20, False), (11, True), (10, False), (24, True), (4, True)], [(14, True), (21, True), (16, False), (12, False), (6, False)]]]
Winner:  Board: [[(14, True), (21, True), (17, True), (24, True), (4, True)], [(10, False), (16, False), (15, False), (9, True), (19, False)], [(18, False), (8, False), (23, True), (26, False), (20, False)], [(22, False), (11, True), (13, False), (6, Fals

In [99]:
bingo_test = Bingo(bingo_text)
loser, number = bingo_test.draw_until_last()

print("Loser: ", loser)
print(loser.sum_unmarked, number, number*loser.sum_unmarked)


Loser:  Board: [[(3, False), (15, False), (0, True), (2, True), (22, False)], [(9, True), (18, False), (13, True), (17, True), (5, True)], [(19, False), (8, False), (7, True), (25, False), (23, True)], [(20, False), (11, True), (10, True), (24, True), (4, True)], [(14, True), (21, True), (16, True), (12, False), (6, False)]]
148 13 1924


In [83]:
bingo = Bingo.from_file('4-puzzle-input.txt')
winner, number = bingo.draw_until_win()

print("Winner: ", winner)
print(winner.sum_unmarked, number, number*winner.sum_unmarked)


## Day 4 part 2

In [100]:
bingo = Bingo.from_file('4-puzzle-input.txt')
winner, number = bingo.draw_until_last()

print("Loser: ", winner)
print(winner.sum_unmarked, number, number*winner.sum_unmarked)


Loser:  Board: [[(45, False), (97, True), (96, True), (23, True), (62, True)], [(79, True), (59, True), (60, True), (87, True), (64, False)], [(75, True), (2, False), (30, True), (47, True), (50, True)], [(85, True), (81, True), (56, True), (11, False), (38, True)], [(17, True), (26, True), (40, True), (7, True), (66, True)]]
122 40 4880


## Day 5 part 1

In [29]:
from dataclasses import dataclass
import re
from collections import Counter


def intermediates(n1: int, n2: int) -> List[int]:
    s = 1 if n1 < n2 else -1
    return [n for n in range(n1, n2 + s, s)]


@dataclass(unsafe_hash=True, order=True)
class Point:
    x: int
    y: int


@dataclass
class Line:
    start: Point
    end: Point

    @classmethod
    def from_text(cls, data: str):
        m = re.match(r'(\d+),(\d+) -> (\d+),(\d+)', data)
        return cls(start=Point(int(m[1]), int(m[2])), end=Point(int(m[3]), int(m[4])))

    @property
    def points(self):
        if self.start.x == self.end.x or self.start.y == self.end.y:
            r_x = sorted([self.start.x, self.end.x])
            r_x[-1] += 1
            r_y = sorted([self.start.y, self.end.y])
            r_y[-1] += 1
            return [Point(x, y) for x in range(*r_x) for y in range(*r_y)]
        elif (abs(self.start.x - self.end.x) == abs(self.start.y - self.end.y)):
            vals = zip(intermediates(self.start.x, self.end.x),
                       intermediates(self.start.y, self.end.y))
            return [Point(x, y) for x, y in vals]
        else:
            return []


l = Line.from_text('3,4 -> 1,4')
print(l.points)
print(Line.from_text('3,6 -> 2,3').points)
print(Line.from_text('3,6 -> 3,3').points)
print(Line.from_text('3,3 -> 5,5').points)
print(Line.from_text('6,6 -> 4,4').points)
print(Line.from_text('8,0 -> 0,8').points)


[Point(x=1, y=4), Point(x=2, y=4), Point(x=3, y=4)]
[]
[Point(x=3, y=3), Point(x=3, y=4), Point(x=3, y=5), Point(x=3, y=6)]
[Point(x=3, y=3), Point(x=4, y=4), Point(x=5, y=5)]
[Point(x=6, y=6), Point(x=5, y=5), Point(x=4, y=4)]
[Point(x=8, y=0), Point(x=7, y=1), Point(x=6, y=2), Point(x=5, y=3), Point(x=4, y=4), Point(x=3, y=5), Point(x=2, y=6), Point(x=1, y=7), Point(x=0, y=8)]


In [30]:
class Grid:
    def __init__(self, lines:List[Line]) -> None:
        self.points = Counter()
        self.lines = []
        for line in lines:
            self.add_line(line)

    def add_line(self, line):
        self.lines.append(line)
        self.points.update(line.points)

    @classmethod
    def from_file(cls, filename):
        with Path(filename) as file:
            lines = [Line.from_text(l) for l in file.read_text().splitlines()]

        return cls(lines)

    @property
    def dangerous_points(self):
        return[x[0] for x in self.points.most_common() if x[1] > 1]

    def __repr__(self) -> str:
        s_x = max(p.x for p in self.points) + 1
        s_y = max(p.y for p in self.points) + 1
        grid = [['.']*s_x for _ in range(s_y)]
        for p, v in self.points.items():
            grid[p.y][p.x] = str(v)

        rv = "\n".join(''.join(l) for l in grid)
        return rv + "\n" + str(self.points)

    def __str__(self) -> str:
        return self.__repr__()
            

g = Grid.from_file('5-test.txt')
print(g.lines)
print(g)

[Line(start=Point(x=0, y=9), end=Point(x=5, y=9)), Line(start=Point(x=8, y=0), end=Point(x=0, y=8)), Line(start=Point(x=9, y=4), end=Point(x=3, y=4)), Line(start=Point(x=2, y=2), end=Point(x=2, y=1)), Line(start=Point(x=7, y=0), end=Point(x=7, y=4)), Line(start=Point(x=6, y=4), end=Point(x=2, y=0)), Line(start=Point(x=0, y=9), end=Point(x=2, y=9)), Line(start=Point(x=3, y=4), end=Point(x=1, y=4)), Line(start=Point(x=0, y=0), end=Point(x=8, y=8)), Line(start=Point(x=5, y=5), end=Point(x=8, y=2))]
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
Counter({Point(x=4, y=4): 3, Point(x=6, y=4): 3, Point(x=0, y=9): 2, Point(x=1, y=9): 2, Point(x=2, y=9): 2, Point(x=7, y=1): 2, Point(x=5, y=3): 2, Point(x=3, y=4): 2, Point(x=7, y=4): 2, Point(x=2, y=2): 2, Point(x=7, y=3): 2, Point(x=5, y=5): 2, Point(x=3, y=9): 1, Point(x=4, y=9): 1, Point(x=5, y=9): 1, Point(x=8, y=0): 1, Point(x=6, y=2): 1, Point(x=3, y=5): 1, Point(x=2, y=6): 1, 

In [31]:
day5_grid = Grid.from_file('5-data.txt')

print(len(day5_grid.dangerous_points))

21373


## Day 5 part 2
Point.points aangepast

In [32]:
Grid.from_file('5-test.txt').dangerous_points

[Point(x=4, y=4),
 Point(x=6, y=4),
 Point(x=0, y=9),
 Point(x=1, y=9),
 Point(x=2, y=9),
 Point(x=7, y=1),
 Point(x=5, y=3),
 Point(x=3, y=4),
 Point(x=7, y=4),
 Point(x=2, y=2),
 Point(x=7, y=3),
 Point(x=5, y=5)]